In [1]:
import numpy as np
import torch 
from pathlib import Path
import plotly.graph_objects as go

from gpmdm import GPMDM
from gpmdm_pf import GPMDM_PF
from dataset_utils.mocap_labels import WALK_TRIALS_TRAIN, WALK_TRIALS_TEST, RUN_TRIALS_TRAIN, RUN_TRIALS_TEST
import dataset_utils.select_joints as select_joints

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.6.1 (SDL 2.30.7, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
config_path = Path.cwd() / 'models' / 'gpmdm' / 'gpmdm_config.pth'
state_path = Path.cwd() / 'models' / 'gpmdm' / 'gpmdm_state.pth'

In [3]:
reduced_columns = WALK_TRIALS_TEST[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")


Number of DOFs: 35


In [4]:
TEST_TRIALS = RUN_TRIALS_TEST + WALK_TRIALS_TEST
TRUTH_LABELS = [1] * len(RUN_TRIALS_TEST) + [0] * len(WALK_TRIALS_TEST)

reduced_columns = TEST_TRIALS[0].get_columns_for_joints(select_joints.WALKING_SIMPLIFIED_JOINTS)
DOFs = len(reduced_columns)
print(f"Number of DOFs: {DOFs}")

Number of DOFs: 35


In [5]:
d = 3 # latent space dimension
DOFs = len(reduced_columns)  # number of degrees of freedom

dyn_back_step = 1 # Number of time steps to look back in the dynamics GP

# Initial values for hyperparameters
y_lambdas_init = np.ones(DOFs)  # Signal standard deviation for observation GP
y_lengthscales_init = np.ones(d)  # Lengthscales for observation GP
y_sigma_n_init = 1e-2  # Noise standard deviation for observation GP

x_lambdas_init = np.ones(d)  # Signal standard deviation for latent dynamics GP
x_lengthscales_init = np.ones(dyn_back_step*d)  # Lengthscales for latent dynamics GP
x_sigma_n_init = 1e-2  # Noise standard deviation for latent dynamics GP
x_lin_coeff_init = np.ones(dyn_back_step*d + 1)  # Linear coefficients for latent dynamics GP


In [6]:
gpdm = GPMDM(
        D=DOFs,
        d=d,
        n_classes=2,
        dyn_target='full',
        dyn_back_step=dyn_back_step,
        y_lambdas_init=y_lambdas_init,
        y_lengthscales_init=y_lengthscales_init,
        y_sigma_n_init=y_sigma_n_init,
        x_lambdas_init=x_lambdas_init,
        x_lengthscales_init=x_lengthscales_init,
        x_sigma_n_init=x_sigma_n_init,
        x_lin_coeff_init=x_lin_coeff_init
    )

In [7]:
gpdm.load(config_path, state_path)


GPDM correctly loaded


In [8]:
# Markov Transition Matrix between classes
markov_transition_matrix = torch.tensor(
    [[0.9, 0.1], 
     [0.1, 0.9]])

# Build a gpdm_pf model
gpdm_pf = GPMDM_PF(gpdm, 
                   markov_switching_model=markov_transition_matrix,
                   num_particles=100,
                   n_fresh_particles_at_each_timestep=10)
   

In [9]:
trial_number = 23
test_trial = TEST_TRIALS[trial_number]
test_trial_label = TRUTH_LABELS[trial_number]
data_arr = test_trial.as_dataframe()[reduced_columns].to_numpy(dtype=np.float32)[::5, :]

In [10]:
gpdm_pf.reset()
correct = 0
total = 0

predicted_latent_states = torch.zeros((len(data_arr)+1, d))
predicted_latent_states[0] = gpdm_pf.current_state_mean()

# Run the filter bank on the trial
for data in data_arr:
    gpdm_pf.update(data)
    predicted_label = gpdm_pf.get_most_likely_class()
    latest_lls = gpdm_pf.class_probabilities()
    walk_ll = latest_lls[0]
    run_ll = latest_lls[1]
    predicted_latent_states[total] = gpdm_pf.current_state_mean()

    if predicted_label == test_trial_label:
        correct += 1
    
    total += 1

accuracy = correct / total

print(f"True label: {test_trial_label}, Predicted label: {predicted_label}")
print(f"Accuracy: {accuracy}")

True label: 0, Predicted label: 1
Accuracy: 0.12048192771084337


In [11]:

# Assuming gpdm.get_X_for_class is your method to get the data
X0 = gpdm.get_X_for_class(0).detach().numpy()
X1 = gpdm.get_X_for_class(1).detach().numpy()
predicted_latent_states = predicted_latent_states.detach().numpy()

# Create the plot
fig = go.Figure()

# Add Class 0 data (in blue)
fig.add_trace(go.Scatter3d(
    x=X0[:, 0],
    y=X0[:, 1],
    z=X0[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='Class 0'
))

# Add Class 1 data (in red)
fig.add_trace(go.Scatter3d(
    x=X1[:, 0],
    y=X1[:, 1],
    z=X1[:, 2],
    mode='markers',
    marker=dict(size=5, color='red'),
    name='Class 1'
))

# Add predicted latent states (in green)
# make then a gradient of green to show the trajectory
fig.add_trace(go.Scatter3d(
    x=predicted_latent_states[:, 0],
    y=predicted_latent_states[:, 1],
    z=predicted_latent_states[:, 2],
    mode='lines+markers',
    marker=dict(size=5, color=np.linspace(0, 1, len(predicted_latent_states)), colorscale='Greens'),
    line=dict(color='green', width=2),
    name='Predicted Latent States'
))

# Update layout with axis labels and title
fig.update_layout(
    title="3D Latent Space Plot",
    scene=dict(
        xaxis_title="X-axis",
        yaxis_title="Y-axis",
        zaxis_title="Z-axis"
    )
)

# Show the interactive plot
fig.show()